In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Example process of Sentinel-1 GRD ODC Prep
- Outputs backscatter (Gamma-VV/VH) products and layover and shadow masks
- Requires S3-like bucket hosted DEM
- Assumes uploaded to S3-like bucket
- Will try Alaska Satellite Facility then ESA Open Access Hub
- Aligned w/ CEOS ARD Definition but not validated as meeting any brand standard

Would like to...
- add option to scale (*100) and define precision (4 *sf*) of output db values so can convert dtype to uint16 (float64 takes up spacceeee) 
- amend snap process to inc. more robust no data value (currently uses 0...)
- make creation of layover+shadow mask optional (almost doubles processing as requires alternate (simulater) terrain correction)
- expand to inc. other sources of pre-generated ARD
- align more explicitly w/ CEOS ARD def/standard

In [26]:
import os
import time
from tqdm import tqdm

from utils.prepS1 import prepareS1

In [27]:
source_envars = ['AWS_ACCESS_KEY_ID','AWS_SECRET_ACCESS_KEY','AWS_S3_ENDPOINT_URL','COPERNICUS_USERNAME','COPERNICUS_PWD','S1_PROCESS_P1A','S1_PROCESS_P2A','S1_PROCESS_P3A','S1_PROCESS_P4A','SNAP_GPT','ASF_USERNAME','ASF_PWD']
for var in source_envars:
    print(var,':',os.getenv(var)!=None)

AWS_ACCESS_KEY_ID : True
AWS_SECRET_ACCESS_KEY : True
AWS_S3_ENDPOINT_URL : True
COPERNICUS_USERNAME : True
COPERNICUS_PWD : True
S1_PROCESS_P1A : True
S1_PROCESS_P2A : True
S1_PROCESS_P3A : True
S1_PROCESS_P4A : True
SNAP_GPT : True
ASF_USERNAME : True
ASF_PWD : True


In [28]:
# os.getenv('LOGLEVEL')
os.environ["LOGLEVEL"] = 'DEBUG'

In [29]:
scenes = ['S1A_IW_GRDH_1SDV_20200117T064006_20200117T064041_030836_0389BE_D8FE']
scenes = ['S1A_IW_GRDH_1SSV_20141020T173958_20141020T174027_002916_0034E8_DA92'] # this is one that errored out alread
scenes = [
    'S1A_IW_GRDH_1SSV_20141020T173958_20141020T174027_002916_0034E8_DA92', # bnd
    'S1A_IW_GRDH_1SSV_20151209T063939_20151209T064015_008961_00CD77_6703', # asf
    'S1A_IW_GRDH_1SSV_20151214T173958_20151214T174027_009041_00CF93_0E88', # asf
    'S1A_IW_GRDH_1SSV_20150217T173955_20150217T174024_004666_005C26_68DA', # bnd
    'S1A_IW_GRDH_1SSV_20170101T174003_20170101T174032_014641_017CE8_8CC3', # bnd
    'S1A_IW_GRDH_1SDV_20200415T174051_20200415T174114_032141_03B737_97F2' # ok
         ]

scenes = [
    'S1B_IW_GRDH_1SDV_20200131T071248_20200131T071317_020057_025F57_31A8' # Vanuatu
]

In [30]:
%%time
# Vanuatu
for scene in tqdm(scenes[:]):
    print('')
    prepareS1(scene, 
              ext_dem='common_sensing/ancillary_products/SRTM1Sec/Vanuatu_DEM.tif',
              s3_bucket='public-eo-data', 
              s3_dir='fiji/tom/sentinel1_corrections_VA/', 
              inter_dir='/tmp/data/S1_ARD/')




  0%|          | 0/1 [00:00<?, ?it/s]INFO:root:S1B_IW_GRDH_1SDV_20200131T071248_20200131T071317_020057_025F57_31A8.SAFE S1B_IW_GRDH_1SDV_20200131T071248 Starting
INFO:root:S1B_IW_GRDH_1SDV_20200131T071248_20200131T071317_020057_025F57_31A8.SAFE S1B_IW_GRDH_1SDV_20200131T071248 DOWNLOADING via ASF
INFO:root:fetching: https://api.daac.asf.alaska.edu/services/search/param?granule_list=S1B_IW_GRDH_1SDV_20200131T071248_20200131T071317_020057_025F57_31A8&output=csv


INFO:root:logging into asf
INFO:root:successfully logged into asf
INFO:root:Extracting ASF scene: /tmp/data/S1_ARD/S1B_IW_GRDH_1SDV_20200131T071248_tmp/S1B_IW_GRDH_1SDV_20200131T071248_20200131T071317_020057_025F57_31A8.zip
INFO:root:S1B_IW_GRDH_1SDV_20200131T071248_20200131T071317_020057_025F57_31A8.SAFE S1B_IW_GRDH_1SDV_20200131T071248 DOWNLOADED via ASF
INFO:root:['/opt/snap/bin/gpt', '/opt/notebooks/utils/cs_s1_pt1_bnr_Orb_Cal_ML.xml', '-Pinput_grd=/tmp/data/S1_ARD/S1B_IW_GRDH_1SDV_20200131T071248_tmp/S1B_IW_GRDH_1SDV_20200131T071248_20200131T071317_020057_025F57_31A8.SAFE/manifest.safe', '-Poutput_ml=/tmp/data/S1_ARD/S1B_IW_GRDH_1SDV_20200131T071248_tmp/S1B_IW_GRDH_1SDV_20200131T071248_Orb_Cal_Deb_ML.dim']
INFO:snap_stderr:stderr:INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO:snap_stderr:stderr:INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INF

not boo





100%|██████████| 1/1 [55:27<00:00, 3327.29s/it]

CPU times: user 33min 9s, sys: 5min 35s, total: 38min 45s
Wall time: 55min 27s


In [24]:
%%time
# Fiji
for scene in tqdm(scenes[:]):
    print('')
    prepareS1(scene, 
              ext_dem='common_sensing/ancillary_products/SRTM1Sec/SRTM30_Fiji_E.tif',
              s3_bucket='public-eo-data', 
              s3_dir='fiji/tom/sentinel1_corrections/', 
              inter_dir='/tmp/data/S1_ARD/')




  0%|          | 0/6 [00:00<?, ?it/s]INFO:root:S1A_IW_GRDH_1SSV_20141020T173958_20141020T174027_002916_0034E8_DA92.SAFE S1A_IW_GRDH_1SSV_20141020T173958 Starting
INFO:root:S1A_IW_GRDH_1SSV_20141020T173958_20141020T174027_002916_0034E8_DA92.SAFE S1A_IW_GRDH_1SSV_20141020T173958 DOWNLOADING via ASF
INFO:root:fetching: https://api.daac.asf.alaska.edu/services/search/param?granule_list=S1A_IW_GRDH_1SSV_20141020T173958_20141020T174027_002916_0034E8_DA92&output=csv


INFO:root:logging into asf
INFO:root:successfully logged into asf
INFO:root:Extracting ASF scene: /tmp/data/S1_ARD/S1A_IW_GRDH_1SSV_20141020T173958_tmp/S1A_IW_GRDH_1SSV_20141020T173958_20141020T174027_002916_0034E8_DA92.zip
INFO:root:S1A_IW_GRDH_1SSV_20141020T173958_20141020T174027_002916_0034E8_DA92.SAFE S1A_IW_GRDH_1SSV_20141020T173958 DOWNLOADED via ASF
INFO:root:['/opt/snap/bin/gpt', '/opt/notebooks/utils/cs_s1_pt1_bnr_Orb_Cal_ML.xml', '-Pinput_grd=/tmp/data/S1_ARD/S1A_IW_GRDH_1SSV_20141020T173958_tmp/S1A_IW_GRDH_1SSV_20141020T173958_20141020T174027_002916_0034E8_DA92.SAFE/manifest.safe', '-Poutput_ml=/tmp/data/S1_ARD/S1A_IW_GRDH_1SSV_20141020T173958_tmp/S1A_IW_GRDH_1SSV_20141020T173958_Orb_Cal_Deb_ML.dim']


True
/tmp/data/S1_ARD/S1A_IW_GRDH_1SSV_20141020T173958_tmp/S1A_IW_GRDH_1SSV_20141020T173958_Orb_Cal_Deb_ML.dim


INFO:snap_stderr:stderr:INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO:snap_stderr:stderr:INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO:snap_stdout:Executing processing graph
INFO:snap_stderr:stderr:INFO: org.hsqldb.persist.Logger: dataFileCache open start
INFO:snap_stdout:version = 2.36
INFO:snap_stdout:....10%....20%....30%....40%....50%....60%....70%....80%....90% done.
INFO:root:S1A_IW_GRDH_1SSV_20141020T173958_20141020T174027_002916_0034E8_DA92.SAFE S1A_IW_GRDH_1SSV_20141020T173958 PROCESSED to MULTILOOK starting PT2
INFO:root:['/opt/snap/bin/gpt', '/opt/notebooks/utils/cs_s1_pt2A_TF.xml', '-Pinput_ml=/tmp/data/S1_ARD/S1A_IW_GRDH_1SSV_20141020T173958_tmp/S1A_IW_GRDH_1SSV_20141020T173958_Orb_Cal_Deb_ML.dim', '-Pext_dem=/tmp/data/S1_ARD/S1A_IW_GRDH_1SSV_20141020T173958_tmp/ext_dem.tif', '-Poutput_tf=/tmp/data/S1_ARD/S1A_IW_GRDH_1SSV_2014102

not boo


...

True
/tmp/data/S1_ARD/S1A_IW_GRDH_1SDV_20200415T174051_tmp/S1A_IW_GRDH_1SDV_20200415T174051_Orb_Cal_Deb_ML.dim


INFO:snap_stderr:stderr:INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO:snap_stderr:stderr:INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO:snap_stdout:Executing processing graph
INFO:snap_stderr:stderr:INFO: org.hsqldb.persist.Logger: dataFileCache open start
INFO:snap_stdout:version = 3.2
INFO:snap_stderr:stderr:INFO: org.esa.snap.engine_utilities.download.DownloadableContentImpl: http retrieving http://aux.sentinel1.eo.esa.int/POEORB/2020/05/05/S1A_OPER_AUX_POEORB_OPOD_20200505T120837_V20200414T225942_20200416T005942.EOF
INFO:snap_stdout:....10%....20%....30%....40%....50%....60%....70%....80%....90% done.
INFO:root:S1A_IW_GRDH_1SDV_20200415T174051_20200415T174114_032141_03B737_97F2.SAFE S1A_IW_GRDH_1SDV_20200415T174051 PROCESSED to MULTILOOK starting PT2
INFO:root:['/opt/snap/bin/gpt', '/opt/notebooks/utils/cs_s1_pt2A_TF.xml', '-Pinput_ml=/tmp

not boo





100%|██████████| 6/6 [2:53:43<00:00, 1737.28s/it]

CPU times: user 1h 57min 50s, sys: 17min 10s, total: 2h 15min
Wall time: 2h 53min 43s
